In [2]:
import pandas as pd
import time 
import pickle
import os
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

In [3]:
#import pre-processed data from pickle 
#df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_text_wo_names.pickle")
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2011_text_wo_names.pickle")
#df_12= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2012_text_wo_names.pickle")
#df_13= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2013_text_wo_names.pickle")
#df_14= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2014_text_wo_names.pickle")
#df_15= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2015_text_wo_names.pickle")
#df_16= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2016_text_wo_names.pickle")
#df_17= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2017_text_wo_names.pickle")
#df_18= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2018_text_wo_names.pickle")
#df_19 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2019_text_wo_names.pickle")
#df_20 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2020_text_wo_names.pickle")
#df_21 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2021_text_wo_names.pickle")
df_22 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2022_text_wo_names_(1).pickle")
df_22

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
6,"[add, three, decade, building, extremely, prou...",1,0,"[Balhousie, Care]",True,"He added: ""After three decades of building Bal...",13,2022,0
10,"[hospitalise, number, facial, fracture, follow...",2,0,[],None,A man in his 20s has been hospitalised with a ...,24,2022,0
12,"[victim, front, passenger, seat, vehicle, atta...",5,0,[],None,The victim was in the front passenger seat of ...,24,2022,0
14,"[involve, bitter, dispute, organisation, membe...",0,1,[Baroness],None,"Baroness Scotland, Secretary General of the Co...",30,2022,1
19,"[add, chair, regret, challenge, position, take...",1,0,[],None,"""He adds: ""The chair regrets and challenges th...",30,2022,0
...,...,...,...,...,...,...,...,...,...
1412503,"[ground, hard, thaw, yet, come, experience, sh...",1,0,[],None,"""The ground has been very hard, the thaw is ye...",1043999,2022,0
1412507,"[endeavour, get, ahead, game, best, find, situ...",1,0,[],None,"""What we are endeavouring to do here is get ah...",1043999,2022,0
1412512,"[die, hit, garage, forecourt]",1,0,[],None,A man has died after being hit by a 4x4 on a g...,1044006,2022,0
1412513,"[north, say, incident, garage, happen, area, t...",2,0,"[Wales, Chester]",True,North Wales Police said the incident at the Pr...,1044006,2022,0


In [4]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [5]:
#apply function to only get rows with an absolute count 
df_22['col_type'] = df_22.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_22 = df_22[df_22["col_type"].notnull()]

#DOC: number of male and female columns
df_22["col_type"].value_counts()  

0.0    250053
1.0    138572
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [7]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [ ]:
#word2vec to dos
#import gensim
#from gensim.models import Word2Vec
#file_w2vec= (r"/Users/yolandaferreirofranchi/Desktop/GoogleNews-vectors-negative300.bin") #yolanda's path 
#model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_w2vec, binary=True)

In [6]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag' 'saga'], #removed liblinear as it is for small + medium datasets & NOT for sparse data
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    #results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Accuracy: {accuracy:.2f}")
    #print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:200]
    low_coef = sorted_coef[-200:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #write word probability list
    #feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    #probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    #positive_probas = probas[:, 1]
    #probabilities = [None] * len(feature_indices)
    #for feature, index in feature_indices.items():
        #probabilities[index] = f"{feature}: {(X_transformed[:, index].toarray() * positive_probas).mean()}"

    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, best_clf, class_report #df_probs_top

**Coefficient Analysis**

In [7]:
df_probs_22 = logistic_regression_year(df_22, 'col_type', 'pre_processed_sent')
df_probs_22


Execution time: 359.07 seconds


(           feature      coef
 0         pregnant  6.744265
 1        pregnancy  5.094695
 2            hijab  4.824436
 3        boyfriend  4.818898
 4        maternity  4.793203
 ..             ...       ...
 195     menstruate  1.772014
 196       greville  1.767179
 197  congresswoman  1.762240
 198          adele  1.761328
 199           lipa  1.755909
 
 [200 rows x 2 columns],
          feature      coef
 0           dems -1.616377
 1      monastery -1.617422
 2          shirt -1.617750
 3        feather -1.618123
 4         thomas -1.618988
 ..           ...       ...
 195     chairman -3.416181
 196          iii -3.600857
 197     prostate -3.642648
 198  businessman -3.645008
 199           tï -5.968579
 
 [200 rows x 2 columns],
 LogisticRegression(C=1, class_weight='balanced', random_state=42,
                    solver='newton-cg'),
 '              precision    recall  f1-score   support\n\n         0.0       0.76      0.65      0.70     49956\n         1.0       0.50     

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [8]:
#open model performance metrics 
with open('results_22.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification report:\n{report}")

Accuracy: 0.64
Classification report:
              precision    recall  f1-score   support

         0.0       0.76      0.65      0.70     49956
         1.0       0.50      0.63      0.56     27769

    accuracy                           0.64     77725
   macro avg       0.63      0.64      0.63     77725
weighted avg       0.67      0.64      0.65     77725



In [10]:
#create DF of highest coef
highest_coef_22 = pd.DataFrame(df_probs_22[0])
highest_coef_22["coef_type"] = "highest"
highest_coef_22["year"] = 2022

#create DF of lowest lowest coef manipulation 
lowest_coef_22 = pd.DataFrame(df_probs_22[1]) 
lowest_coef_22 = lowest_coef_22.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_22["coef_type"] = "lowest" #coef type
lowest_coef_22["year"] = 2022 #year 
highest_coef_22.head(50)

,feature,coef,coef_type
0,pregnant,6.744265,highest
1,pregnancy,5.094695,highest
2,hijab,4.824436,highest
3,boyfriend,4.818898,highest
4,maternity,4.793203,highest
5,reign,4.662568,highest
6,abortion,4.530159,highest
7,headscarf,4.475117,highest
8,female,4.244700,highest
9,midwife,4.191776,highest


In [11]:
#save DF as pickle file per year 
lowest_coef_22.to_pickle('RESULTS22_coef_low.pickle')
highest_coef_22.to_pickle('RESULTS22_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Word Probability Analysis** 

#make sure to un-comment the last two lines of the functions

In [ ]:
#access list of top words 
#word_prob_10 = pd.DataFrame(df_probs_10[3]) 
#word_prob_10 = word_prob_10.sort_values(["probability"], ascending= False)

#save list of words (highest to lowest probability of being female) for year: 
#word_prob_10.to_pickle('RESULTS09_words_prob.pickle')
#word_prob_10.head(10)

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

In [15]:
def load_pickle_files_low(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("low.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

def load_pickle_files_high(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("high.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

In [18]:
#stack dfs together for low and high coefs 
df_low = load_pickle_files_low(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high = load_pickle_files_high(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high

list

**Support Vector Machine**

In [6]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
def svm_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'C': [.01, .1, 1, 10, 100],
        #'kernel': default('rbf')',
        'degree': [2,3,4],
        'gamma': ['scale', 'auto'],
        #'class_weight': [None],
        #'random_state': [42]
    }

    #the classifier 
    clf = SVC()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = SVC(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #get important features (equivalent of coefficients in logreg)
    result_clf = permutation_importance(best_clf, X_test, y_test, n_repeats=10, random_state=42)
    importance_scores = result_clf.importances_mean
    importance_scores_df = pd.DataFrame(importance_scores, columns=['tbd'])

    for i in range(len(importance_scores)):
        print("Feature {}: Importance score = {:.3f}".format(i, importance_scores[i]))


    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return importance_scores_df

In [8]:
svm_year(df_22, 'col_type', 'pre_processed_sent') #takes more than 1 hour to run

: 

: 

In [ ]:
#save the SVM model 
import joblib
model, accuracy, class_report = svm_year(df_09, 'col_type', 'pre_processed_sent')

# Save the model and performance metrics
joblib.dump(model, 'model_svm.pkl')
with open('performance_metrics.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Classification Report:\n{class_report}")

**RNN** 

In [27]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def rnn(df, sentence_col, label_col):
    # Tokenize sentences
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df[sentence_col])
    sequences = tokenizer.texts_to_sequences(df[sentence_col])

    # Define parameters
    vocab_size = len(tokenizer.word_index) + 1
    max_len = 100
    embedding_dim = 50

    # Pad sequences
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

    # Split into training and testing sets
    # Convert label column to numeric values
    le = LabelEncoder()
    df[label_col] = le.fit_transform(df[label_col])

# Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df[sentence_col], df[label_col], test_size=0.2, random_state=42)

    # Define RNN model
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
    model.add(LSTM(units=64))
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    return model


In [28]:
rnn(df_22,'pre_processed_sent', 'col_type')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).